In [ ]:
def dy_dx(x):
  return 2*x

In [ ]:
dy_dx(3)

6

In [ ]:
import torch

In [ ]:
x = torch.tensor(3.0 ,requires_grad=True)

In [ ]:
y = x**2

In [ ]:
x

tensor(3., requires_grad=True)

In [ ]:
y

tensor(9., grad_fn=<PowBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad


tensor(6.)

In [ ]:
import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [ ]:
dz_dx(4)

-7.661275842587077

In [ ]:
x = torch.tensor(4.0, requires_grad=True)

In [ ]:
y = x ** 2

In [ ]:
z = torch.sin(y)

In [ ]:
x

tensor(4., requires_grad=True)

In [ ]:
y

tensor(16., grad_fn=<PowBackward0>)

In [ ]:
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [ ]:
z.backward()

In [ ]:
x.grad

tensor(-7.6613)

In [ ]:
y.grad # grad not on intermidiate nodes

/tmp/ipython-input-3703736803.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad # grad not on intermidiate nodes


In [ ]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [ ]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [ ]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [ ]:
loss

tensor(6.7012)

In [ ]:
# backpropagation
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [ ]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


Solving entire above process by Autograd

In [ ]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

# note : we have not put autograd = True. This is because we dont have any diiferentiations wrto x and y in this case

In [ ]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [ ]:
w

tensor(1., requires_grad=True)

In [ ]:
b

tensor(0., requires_grad=True)

In [ ]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [ ]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [ ]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [ ]:
loss.backward()

In [ ]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


Other uses of Autograd

In [ ]:
x = torch.tensor([1.0,2.0,3.0] ,requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [ ]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [ ]:
# clearing grad
x = torch.tensor(2.0,requires_grad=True)
x

tensor(2., requires_grad=True)

In [ ]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad      # issue of gradient accumulation as we run multiple forward pass

tensor(4.)

In [ ]:
x.grad.zero_()  # resets gradient

tensor(0.)

In [ ]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

In [ ]:
x.requires_grad_(False)
x

tensor(2.)

In [ ]:
y = x**2
y                 # no gradient function attribute

tensor(4.)

*Detach*

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [ ]:
z = x.detach() # not z is not a part of computational graph, thus cannot do backprop
z

tensor(2.)

In [ ]:
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [ ]:
y1 = z ** 2
y1

tensor(4.)

In [ ]:
y.backward()

In [ ]:
y1.backward()     # confirms backprop not possibl

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Use no-grad function

In [ ]:
with torch.no_grad():
  y = x**2
  print(y)   # backward not working

tensor(4.)
